In [18]:
import os
import gzip
import shutil
from tqdm import tqdm
import zipfile
import subprocess
import sys
import pandas as pd
import cv2
import numpy as np
import copy
import torch
import matplotlib.pyplot as plt

In [2]:
def unzip(data_root, target_root):
  for item in tqdm(os.listdir(data_root)):
    with zipfile.ZipFile(data_root + item, "r") as zip_ref:
      zip_ref.extractall(target_root)

In [3]:
target_root = 'C:\\Users\\Admin\\Desktop\\KohanovDiploma\\masksBrats_new\\'
os.mkdir(target_root)
for item in os.listdir('C:\\Users\\Admin\\Downloads\\masksBrats'):
    os.chdir(target_root)
    os.mkdir(f'{item[:-7]}')
    os.chdir(f'{item[:-7]}')
    shutil.copyfile('C:\\Users\\Admin\\Downloads\\masksBrats\\' + item, item[:-7] + '_seg.nii.gz')

In [22]:
for fold in os.listdir('C:\\Users\\Admin\\Desktop\\KohanovDiploma\\BraTS\\MICCAI_BraTS2020_ValidationData'):
    if not os.path.isfile('C:\\Users\\Admin\\Desktop\\KohanovDiploma\\BraTS\\MICCAI_BraTS2020_ValidationData\\' + fold + f'\\{fold}_seg.nii.gz'):
        shutil.rmtree('C:\\Users\\Admin\\Desktop\\KohanovDiploma\\BraTS\\MICCAI_BraTS2020_ValidationData\\' + fold)
    

In [ ]:
data_root = '/content/drive/Shareddrives/BraTS/'
work_data = '/content/drive/MyDrive/BraTS_for_kirill/'
unzip(data_root, work_data)

100%|██████████| 2/2 [02:29<00:00, 74.83s/it]


In [2]:
import nibabel as nib

In [20]:
def niigz_to_images(data_root, modal):
    for folder in tqdm(os.listdir(data_root)):
        if os.path.isdir(data_root + folder):
            os.chdir(data_root + folder)
            filename = f'{folder}_{modal}.nii.gz'
            if os.path.isfile(filename): 
                images = nib.load(filename).get_data()
                if not os.path.isdir(f'{modal}'):
                    os.mkdir(f'{modal}')
                for j in range(images.shape[2]):
                    image = images[:,:,j]
                    if modal == 'seg':
                        temp = torch.zeros((240,240,3))

                        mask1 = copy.copy(image)
                        mask1 = np.uint8(mask1)
                        mask1 = torch.tensor(mask1)
                        mask1.apply_(lambda x: 1 if x==3 else 0)  # 4

                        mask2 = copy.copy(image)
                        mask2 = np.uint8(mask2)
                        mask2 = torch.tensor(mask2)
                        mask2.apply_(lambda x: 1 if x==1 else 0)  # 2

                        mask3 = copy.copy(image)
                        mask3 = np.uint8(mask3)
                        mask3 = torch.tensor(mask3)
                        mask3.apply_(lambda x: 1 if x==2 else 0)  # 1

                        temp[:,:,0] = mask1
                        temp[:,:,1] = mask2
                        temp[:,:,2] = mask3

                        image = np.asarray(temp)
                        image = image * 255
                    else:
                        rescale = 255/max(np.unique(np.asarray(image)))
                        image = image * rescale
                    cv2.imwrite(data_root + folder + f'/{modal}/{j}.png', image)
                else:
                    continue

In [21]:
data_root = 'C:\\Users\\Admin\\Desktop\\KohanovDiploma\\BraTS\\MICCAI_BraTS2020_TrainingData\\'
val_root = 'C:\\Users\\Admin\\Desktop\\KohanovDiploma\\BraTS\\MICCAI_BraTS2020_ValidationData\\'

In [ ]:
niigz_to_images(val_root, 'seg')

In [ ]:
niigz_to_images(val_root, 't1ce')

In [ ]:
import shutil
data_root = '/content/drive/MyDrive/BraTS/MICCAI_BraTS2020_TrainingData/'
for folder in tqdm(os.listdir(data_root)):
  if os.path.isdir(data_root + folder):
    os.chdir(data_root + folder)
    if os.path.isdir('t1ce'):
      shutil.rmtree('t1ce')




  0%|          | 0/371 [00:00<?, ?it/s]


100%|██████████| 371/371 [00:00<00:00, 3072.23it/s]
